# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 65 new papers today
          23 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/22 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.02510


extracting tarball to tmp_2212.02510...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.02512


extracting tarball to tmp_2212.02512...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.03001


extracting tarball to tmp_2212.03001...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.03215


extracting tarball to tmp_2212.03215...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.02627-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.02627) | **Formation of hub-filament structure triggered by cloud-cloud collision  in W33 complex**  |
|| Jian-Wen Zhou, et al. -- incl., <mark>Shanghuo Li</mark>, <mark>Hong-Li Liu</mark>, <mark>Siju Zhang</mark>, <mark>Chao Zhang</mark>, <mark>Tie Liu</mark>, <mark>Jin-Zeng Li</mark> |
|*Appeared on*| *2022-12-07*|
|*Comments*| *18 pages*|
|**Abstract**| Hub-filament systems are suggested to be birth cradles of high-mass stars and clusters, but the formation of hub-filament structure is still unclear. Using the survey data FUGIN $^{13}$CO (1-0), C$^{18}$O (1-0), and SEDIGISM $^{13}$CO (2-1), we investigate formation of hub-filament structure in W33 complex. W33 complex consists of two colliding clouds, called W33-blue and W33-red. We decompose the velocity structures in W33-blue by fitting multiple velocity components, and find a continuous and monotonic velocity field. Virial parameters of Dendrogram structures suggest the dominance of gravity in W33-blue. The strong positive correlation between velocity dispersion and column density indicates the non-thermal motions in W33-blue may originate from gravitationally driven collapse. These signatures suggest that the filamentary structures in W33-blue result from the gravitational collapse of the compressed layer. However, the large scale velocity gradient in W33-blue may mainly originate from the cloud-cloud collision and feedback of active star formation, instead of the filament-rooted longitudinal inflow. From the above observed results, we argue that the cloud-cloud collision triggers formation of hub-filament structures in W33 complex. Meanwhile, the appearance of multiple-scale hub-filament structures in W33-blue is likely an imprint of the transition from the compressed layer to a hub-filament system. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.02667-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.02667) | **The PHANGS-JWST Treasury Survey: Star Formation, Feedback, and Dust  Physics at High Angular resolution in Nearby GalaxieS**  |
|| <mark>Janice C. Lee</mark>, et al. -- incl., <mark>Eva Schinnerer</mark>, <mark>Frank Bigiel</mark>, <mark>Kiana F. Henny</mark>, <mark>Jonathan D. Henshaw</mark>, <mark>Annie Hughes</mark>, <mark>Kathryn Kreckel</mark>, <mark>Jing Li</mark>, <mark>Daizhong Liu</mark> |
|*Appeared on*| *2022-12-07*|
|*Comments*| *Re-submitted after addressing minor comments from referee. To be published as part of PHANGS-JWST ApJL Focus Issue*|
|**Abstract**| The PHANGS collaboration has been building a reference dataset for the multi-scale, multi-phase study of star formation and the interstellar medium in nearby galaxies. With the successful launch and commissioning of JWST, we can now obtain high-resolution infrared imaging to probe the youngest stellar populations and dust emission on the scales of star clusters and molecular clouds ($\sim$5-50 pc). In Cycle 1, PHANGS is conducting an 8-band imaging survey from 2-21$\mu$m of 19 nearby spiral galaxies. CO(2-1) mapping, optical integral field spectroscopy, and UV-optical imaging for all 19 galaxies have been obtained through large programs with ALMA, VLT/MUSE, and Hubble. PHANGS-JWST enables a full inventory of star formation, accurate measurement of the mass and age of star clusters, identification of the youngest embedded stellar populations, and characterization of the physical state of small dust grains. When combined with Hubble catalogs of $\sim$10,000 star clusters, MUSE spectroscopic mapping of $\sim$20,000 HII regions, and $\sim$12,000 ALMA-identified molecular clouds, it becomes possible to measure the timescales and efficiencies of the earliest phases of star formation and feedback, build an empirical model of the dependence of small dust grain properties on local ISM conditions, and test our understanding of how dust-reprocessed starlight traces star formation activity, all across a diversity of galactic environments. Here we describe the PHANGS-JWST Treasury survey, present the remarkable imaging obtained in the first few months of science operations, and provide context for the initial results presented in the first series of PHANGS-JWST publications. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.02677-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.02677) | **SDSS-IV MaNGA: How Galaxy Interactions Influence Active Galactic Nuclei**  |
|| Joshua L. Steffen, et al. -- incl., <mark>Joel R. Brownstein</mark>, <mark>Arran C. Gross</mark> |
|*Appeared on*| *2022-12-07*|
|*Comments*| *To be published in ApJ. The pair sample is based on the catalog presented in the following work, this https URL*|
|**Abstract**| We present a comparative study of active galactic nuclei (AGN) between galaxy pairs and isolated galaxies with the final data release of the MaNGA integral field spectroscopic survey. We build a sample of 391 kinematic galaxy pairs within the footprint of the survey and select AGN using the survey's spectra. We use the comoving volume densities of the AGN samples to quantify the effects that tidal interactions have on the triggering of nuclear accretion. Our hypothesis is that the pair sample contains AGN that are triggered by not only stochastic accretion but also tidally induced accretion and correlated accretion. With the level of stochastically triggered AGN fixed by the control sample, we model the strength of tidally induced accretion and correlated accretion as a function of projected separation (rp) and compare the model expectations with the observed volume densities of dual AGN and offset AGN (single AGN in a pair). At rp ~ 10 kpc, we find that tidal interactions induce ~30% more AGN than stochastic fueling and cause ~12% of the offset AGN to become dual AGN because of correlations. The strength of both these effects decreases with increasing rp. We also find that the OIII luminosities of the AGN in galaxy pairs are consistent with those found in isolated galaxies, likely because stochastically fed AGN dominate even among close pairs. Our results illustrates that while we can detect tidally induced effects statistically, it is challenging to separate tidally induced AGN and stochastically triggered AGN in interacting galaxies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.02948-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.02948) | **Photometric Mass Estimation and the Stellar Mass-Halo Mass Relation for  Low Mass Galaxies**  |
|| <mark>Dennis Zaritsky</mark>, Peter Behroozi |
|*Appeared on*| *2022-12-07*|
|*Comments*| *Accepted for publication in MNRAS; 13 pages*|
|**Abstract**| We present a photometric halo mass estimation technique for local galaxies that enables us to establish the stellar mass-halo mass (SMHM) relation down to stellar masses of 10$^5$ M$_\odot$. We find no detectable differences among the SMHM relations of four local galaxy clusters or between the cluster and field relations and we find agreement with extrapolations of previous SMHM relations derived using abundance matching approaches. We fit a power law to our empirical SMHM relation and find that for adopted NFW dark matter profiles and for M$_* < 10^9$ M$_\odot$, the halo mass is M$_h = 10^{10.35\pm0.02}({\rm M}_*/10^8 {\rm M}_\odot)^{0.63\pm0.02}$. The normalisation of this relation is susceptible to systematic modelling errors that depend on the adopted dark matter potential and the quoted uncertainties refer to the uncertainties in the median relation. For galaxies with M$_* < 10^{9}$ M$_\odot$ that satisfy our selection criteria, the scatter about the fit in $M_h$, including uncertainties arising from our methodology, is 0.3 dex. Finally, we place lower luminosity Local Group galaxies on the SMHM relationship using the same technique, extending it to M$_* \sim 10^3$ M$_\odot$ and suggest that some of these galaxies show evidence for additional mass interior to the effective radius beyond that provided by the standard dark matter profile. If this mass is in the form of a central black hole, the black hole masses are in the range of intermediate mass black holes, $10^{(5.7\pm0.6)}$ M$_\odot$, which corresponds to masses of a few percent of M$_h$, well above values extrapolated from the relationships describing more massive galaxies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.02960-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.02960) | **Supermassive Black Hole Winds in X-rays -- SUBWAYS. I. Ultra-fast  outflows in QSOs beyond the local Universe**  |
|| G. A. Matzeu, et al. -- incl., <mark>E. Torresi</mark> |
|*Appeared on*| *2022-12-07*|
|*Comments*| *30 pages, 10 figures, accepted for publication in A&A*|
|**Abstract**| We present a new X-ray spectroscopic study of $22$ luminous ($2\times10^{45}\lesssim L_{\rm bol}\rm /erg\,s^{-1} \lesssim 2\times10^{46}$) active galactic nuclei (AGNs) at intermediate-redshift ($0.1 \lesssim z \lesssim 0.4$), as part of the SUpermassive Black hole Winds in the x-rAYS (SUBWAYS) sample, mostly composed of quasars (QSOs) and type\,1 AGN. Here, 17 targets were observed with \textit{XMM-Newton} between 2019--2020 and the remaining 5 are from previous observations. The aim of this large campaign ($1.45\,\rm Ms$ duration) is to characterise the various manifestations of winds in the X-rays driven from supermassive black holes in AGN. In this paper we focus on the search and characterization of ultra-fast outflows (UFOs), which are typically detected through blueshifted absorption troughs in the Fe\,K band ($E>7\,\rm keV$). By following Monte Carlo procedures, we confirm the detection of absorption lines corresponding to highly ionised iron (e.g., Fe\,\textsc{xxv}\,H$\alpha$, Fe\,\textsc{xxvi}\,Ly$\alpha$) in 7/22 sources at the $\gtrsim95\%$ confidence level (for each individual line). The global combined probability of such absorption features in the sample is $>99.9\%$. The SUBWAYS campaign extends at higher luminosity and redshifts than previous local studies on Seyferts, obtained using \xmm and \suzaku observations. We find a UFO detection fraction of $\sim30\%$ on the total sample that is in agreement with the previous findings. This work independently provides further support for the existence of highly-ionised matter propagating at mildly relativistic speed ($\gtrsim0.1c$) in a considerable fraction of AGN over a broad range of luminosities, which is expected to play a key role in the self-regulated AGN feeding-feedback cycle, as also supported by hydrodynamical multiphase simulations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.03001-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.03001) | **Relative astrometry in an annular field**  |
|| Mario Gai, et al. -- incl., <mark>Zhaoxiang Qi</mark> |
|*Appeared on*| *2022-12-07*|
|*Comments*| *20 pages, 16 figures*|
|**Abstract**| Background. Relative astrometry at or below the micro-arcsec level with a 1m class space telescope has been repeatedly proposed as a tool for exo-planet detection and characterization, as well as for several topics at the forefront of Astrophysics and Fundamental Physics. Aim. This paper investigates the potential benefits of an instrument concept based on an annular field of view, as compared to a traditional focal plane imaging a contiguous area close to the telescope optical axis. Method. Basic aspects of relative astrometry are reviewed as a function of the distribution on the sky of reference stars brighter than G = 12 mag (from Gaia EDR3). Statistics of field stars for targets down to G = 8 mag is evaluated by analysis and simulation. Results. Observation efficiency benefits from prior knowledge on individual targets, since source model is improved with few measurements. Dedicated observations (10-20 hours) can constrain the orbital inclination of exoplanets to a few degrees. Observing strategy can be tailored to include a sample of stars, materialising the reference frame, sufficiently large to average down the residual catalogue errors to the desired microarcsec level. For most targets, the annular field provides typically more reference stars, by a factor four to seven in our case, than the conventional field. The brightest reference stars for each target are up to 2 mag brighter. Conclusions. The proposed annular field telescope concept improves on observation flexibility and/or astrometric performance with respect to conventional designs. It appears therefore as an appealing contribution to optimization of future relative astrometry missions. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.03215-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.03215) | **Steady states of the Parker instability**  |
|| Devika Tharakkal, et al. -- incl., <mark>Frederick A. Gent</mark> |
|*Appeared on*| *2022-12-07*|
|*Comments*| **|
|**Abstract**| We study the linear properties, nonlinear saturation and a steady, strongly nonlinear state of the Parker instability in galaxies. We consider magnetic buoyancy and its consequences with and without cosmic rays. Cosmic rays are described using the fluid approximation with anisotropic, non-Fickian diffusion. To avoid unphysical constraints on the instability (such as boundary conditions often used to specify an unstable background state), nonideal MHD equations are solved for deviations from a background state representing an unstable magnetohydrostatic equilibrium. We consider isothermal gas and neglect rotation. The linear evolution of the instability is in broad agreement with earlier analytical and numerical models; but we show that most of the simplifying assumptions of the earlier work do not hold, such that they provide only a qualitative rather than quantitative picture. In its nonlinear stage the instability has significantly altered the background state from its initial state. Vertical distributions of both magnetic field and cosmic rays are much wider, the gas layer is thinner, and the energy densities of both magnetic field and cosmic rays are much reduced. The spatial structure of the nonlinear state differs from that of any linear modes. A transient gas outflow is driven by the weakly nonlinear instability as it approaches saturation. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.02579-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.02579) | **Identifying the 3FHL Catalog. VI. Swift Observations of 3FHL  Unassociated Objects with Source Classification via Machine Learning**  |
|| S. Joffre, et al. -- incl., <mark>N. Torres-Albà</mark> |
|*Appeared on*| *2022-12-07*|
|*Comments*| *15 pages, 11 figures, interactive plot link see, this https URL*|
|**Abstract**| The Third Catalog of Hard Fermi Large Area Telescope Sources (3FHL) reports the detection of 1556 objects at E > 10 GeV. However, 177 sources remain unassociated and 23 are associated with a ROSAT X-ray detection of unknown origin. Pointed X-ray observations were conducted on 30 of these unassociated and unknown sources with Swift-XRT. A bright X-ray source counterpart was detected in 21 out of 30 fields. In five of these 21 fields, we detected more than one X-ray counterpart, totaling 26 X-ray sources analyzed. Multiwavelength data was compiled for each X-ray source detected. We find that 21 out of the 26 X-ray sources detected display the multiwavelength properties of blazars, while one X-ray source displays the characteristics of a Galactic source. Using trained decision tree, random forest, and support vector machine models, we predict all 21 blazar counterpart candidates to be BL Lacertae objects (BL Lacs). This is in agreement with BL Lacs being the most populous source class in the 3FHL. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2212.02627.md
    + _build/html/tmp_2212.02627/./w33/vf.png
    + _build/html/tmp_2212.02627/./w33/vg.png
    + _build/html/tmp_2212.02627/./w33/bts.png
exported in  _build/html/2212.02667.md
    + _build/html/tmp_2212.02667/./Figures/sed_model_filter.png
    + _build/html/tmp_2212.02667/./Figures/judy_n628.png
    + _build/html/tmp_2212.02667/./Figures/judy_ic5332.jpg
    + _build/html/tmp_2212.02667/./Figures/gri_ha_v4.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\msun}{\mbox{M_\odot}}$
$\newcommand{\mean}[1]{\mbox{\langle#1\rangle}}$
$\newcommand{\dif}{\mathrm{d}}$
$\newcommand{\evs}{\color{olive} }$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\msun$}{\mbox{M_\odot}}$
$\newcommand{$\mean$}[1]{\mbox{\langle#1\rangle}}$
$\newcommand{$\dif$}{\mathrm{d}}$
$\newcommand{$\evs$}{\color{olive} }$</div>



<div id="title">

# Formation of hub-filament structure triggered by cloud-cloud collision in W33 complex

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.02627-b31b1b.svg)](https://arxiv.org/abs/2212.02627)<mark>Appeared on: 2022-12-07</mark> - _18 pages_

</div>
<div id="authors">

Jian-Wen Zhou, et al. -- incl., <mark><mark>Shanghuo Li</mark></mark>, <mark><mark>Hong-Li Liu</mark></mark>, <mark><mark>Siju Zhang</mark></mark>, <mark><mark>Chao Zhang</mark></mark>, <mark><mark>Tie Liu</mark></mark>, <mark><mark>Jin-Zeng Li</mark></mark>

</div>
<div id="abstract">

**Abstract:** Hub-filament systems are suggested to be birth cradles of high-mass stars and clusters, but the formation of hub-filament structure is still unclear. Using the survey data FUGIN$^{13}$CO (1-0), C$^{18}$O (1-0), and SEDIGISM$^{13}$CO (2-1), we investigate formation of hub-filament structure in W33 complex.W33 complex consists of two colliding clouds, called W33-blue and W33-red.We decompose the velocity structures in W33-blue by fitting multiple velocity components, and find a continuous and monotonic velocity field. Virial parameters of{\it Dendrogram}structures suggest the dominance of gravity in W33-blue. The strong positive correlation between velocity dispersion and column density indicates the non-thermal motions in W33-blue may originate from gravitationally driven collapse. These signatures suggest that the filamentary structures in W33-blue result from the gravitational collapse of the compressed layer. However, the large scale velocity gradient in W33-blue may mainly originate from the cloud-cloud collision and feedback of active star formation, instead of the filament-rooted longitudinal inflow. From the above observed results, we argue that the cloud-cloud collision triggers formation of hub-filament structures in W33 complex. Meanwhile, the appearance of multiple-scale hub-filament structures in W33-blue is likely an imprint of the transition from the compressed layer to a hub-filament system.

</div>

<div id="div_fig1">

<img src="tmp_2212.02627/./w33/vf.png" alt="Fig4" width="100%"/>

**Figure 4. -** (a) Moment-1 map of $^{13}$CO (1$-$0) for the entire W33 complex, made over the velocity range [29.6, 60.2] km s$^{-1}$. The contour is the \emph{Herschel} H$_{2}$ column density with a level of 3.35$\times$10$^{22}$ cm$^{-2}$, dashed curve marks the bent compression structure due to cloud-cloud collision. Red “+” represents the position of W33-main, while black “+” marks the ATLASGAL clump. Dashed black box marks the bottom of collision crater, where has the most concentrated clump distribution;
(b) The colour images of \emph{Spitzer} 8 $\mu$m emission, black “+” represents the position of W33-main, orange “+” marks the ATLASGAL clump. The red circle shows the IRDC, where the size of the circle reflects the radius of IRDC;
(c) Moment-1 map of $^{13}$CO (1$-$0) for W33-blue over the velocity range [29.6, 43.3] km s$^{-1}$. The contour is the \emph{Herschel} dust temperature with a level of 18.6 K, black “+” marks the ATLASGAL clump. Dashed straight line roughly displays the boundary of two velocity components in W33-blue;
(d) The integrated intensity map of $^{13}$CO (1$-$0), velocity range [29.6, 43.3] km s$^{-1}$. Blue dashed lines display the filament skeletons identified by FILFINDER algorithm according to the integrated intensity map. Red contours show the peak emission of \emph{Spitzer} 8 $\mu$m. Three orange “+” represent O4–7(super)-giant stars, red boxes mark several sub-regions from “1” to “6”, “0” represents the entire region.
(e) The velocity dispersion of W33-blue derived from the moment-2 map of $^{13}$CO (1$-$0) in the velocity range [29.6, 43.3] km s$^{-1}$, black “+” marks the ATLASGAL clump, blue dashed lines display the filament skeletons. Dashed black box marks the bottom of collision crater;
(f) The velocity dispersion of W33-blue derived from the moment-2 map of $^{13}$CO (2$-$1), gray contours show the peak emission of \emph{Spitzer} 8 $\mu$m,  and the black “+” represents O4–7(super)-giant stars. Blue boxes in figures (b), (c) and (f) show the region investigated by \citet{Kohno2018-70S}.
 (*vf*)

</div>
<div id="div_fig2">

<img src="tmp_2212.02627/./w33/vg.png" alt="Fig13" width="100%"/>

**Figure 13. -** (a), (c) and (e): distributions of  normalized velocity and intensity extracted from the moment 0 and moment 1 maps of $^{13}$CO (1$-$0) emission along the filament skeletons marked in Fig.\ref{vf}; (b), (d) and (f): distributions of  normalized velocity and intensity extracted from the moment 0 and moment 1 maps of $^{13}$CO (2$-$1) emission along the filament skeletons. (*vg*)

</div>
<div id="div_fig3">

<img src="tmp_2212.02627/./w33/bts.png" alt="Fig1" width="100%"/>

**Figure 1. -** (a) Average spectra of $^{13}$CO (1-0), C$^{18}$O (1-0) and $^{13}$CO (2-1) for the entire W33 complex;
(b), (c) and (d) Statistical distributions of centroid velocity, velocity dispersion and $\chi^2\rm_c$ value of $^{13}$CO (1-0), C$^{18}$O (1-0) and $^{13}$CO (2-1) line emission fitted by BTS algorithm for W33-blue. (*bts*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\Caltech}{\affil{California Institute of Technology, 1200 E. California Blvd., MC 249-17, Pasadena, CA 91125, USA}}$
$\newcommand{\Tamkang}{\affil{Department of Physics, Tamkang University, No.151, Yingzhuan Rd., Tamsui Dist., New Taipei City 251301, Taiwan}}$
$\newcommand{\GEMINI}{\affil{Gemini Observatory/NSF’s NOIRLab, 950 N. Cherry Avenue, Tucson, AZ, 85719, USA$
$}}$
$\newcommand{\ASCL}{\affil{Astrophysics Source Code Librar$
$y, Michigan Technological University, 1400 Townsend Drive, Houghton, MI 49931}}$
$\newcommand{\OSU}{\affil{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}}$
$\newcommand{\Alberta}{\affil{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}}$
$\newcommand{\ANU}{\affil{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}}$
$\newcommand{\IPARCOS}{\affil{Instituto de F\'{\i}sica de Part\'{\i}culas y del Cosmos, Universidad Complutense de Madrid, E-28040 Madrid, Spain}}$
$\newcommand{\IPAC}{\affil{Caltech-IPAC, 1200 E. California Blvd. Pasadena, CA 91125, USA}}$
$\newcommand{\Carnegie}{\affil{Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}}$
$\newcommand{\CCAPP}{\affil{Center for Cosmology and AstroParticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}}$
$\newcommand{\CfA}{\affil{Center for Astrophysics \mid Harvard \& Smithsonian , 60 Garden Street, Cambridge, MA 02138, USA}}$
$\newcommand{\CITEVA}{\affil{Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}}$
$\newcommand{\CNRS}{\affil{CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\ESO}{\affil{European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei M\"{u}nchen, Germany}}$
$\newcommand{\Heidelberg}{\affil{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}}$
$\newcommand{\COOL}{\affil{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}}$
$\newcommand{\ICRAR}{\affil{International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}}$
$\newcommand{\IRAM}{\affil{Institut de Radioastronomie Millimétrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d'Hères, France}}$
$\newcommand{\IRAP}{\affil{CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\UPS}{\affil{Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\ITA}{\affil{Universit\"{a}t Heidelberg, Zentrum f\"{u}r Astronomie, Institut f\"{u}r Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}}$
$\newcommand{\IWR}{\affil{Universit\"{a}t Heidelberg, Interdisziplin\"{a}res Zentrum f\"{u}r Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}}$
$\newcommand{\JHU}{\affil{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}}$
$\newcommand{\Leiden}{\affil{Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands}}$
$\newcommand{\Maryland}{\affil{Department of Astronomy, University of Maryland, College Park, MD 20742, USA}}$
$\newcommand{\MPE}{\affil{Max-Planck-Institut f\"{u}r extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}}$
$\newcommand{\MPIA}{\affil{Max-Planck-Institut f\"{u}r Astronomie, K\"{o}nigstuhl 17, D-69117, Heidelberg, Germany}}$
$\newcommand{\Nagoya}{\affil{Department of Physics, Nagoya University, Furo-cho, Chikusa-ku, Nagoya, Aichi 464-8602, Japan}}$
$\newcommand{\NRAO}{\affil{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}}$
$\newcommand{\OAN}{\affil{Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}}$
$\newcommand{\ObsParis}{\affil{Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France}}$
$\newcommand{\Princeton}{\affil{Department of Astrophysical Sciences, Princeton University, Princeton, NJ 08544 USA}}$
$\newcommand{\UToledo}{\affil{University of Toledo, 2801 W. Bancroft St., Mail Stop 111, Toledo, OH, 43606}}$
$\newcommand{\Toulouse}{\affil{Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\UBonn}{\affil{Argelander-Institut f\"ur Astronomie, Universit\"at Bonn, Auf dem H\"ugel 71, 53121 Bonn, Germany}}$
$\newcommand{\UChile}{\affil{Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}}$
$\newcommand{\UCM}{\affil{Departamento de F\'{\i}sica de la Tierra y Astrof\'{\i}sica, Universidad Complutense de Madrid, E-28040 Madrid, Spain}}$
$\newcommand{\UCSD}{\affil{Center for Astrophysics and Space Sciences, Department of Physics,  University of California,\San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}}$
$\newcommand{\ULyon}{\affil{Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574,\F-69230 Saint-Genis-Laval, France}}$
$\newcommand{\UMass}{\affil{University of Massachusetts—Amherst, 710 N. Pleasant Street, Amherst, MA 01003, USA}}$
$\newcommand{\UWyoming}{\affil{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}}$
$\newcommand{\LAM}{\affil{$
$Aix Marseille Univ, CNRS, CNES, LAM (Laboratoire d’Astrophysique de Marseille),  F-13388 Marseille,$
$France}}$
$\newcommand{\UHawaii}{\affil{Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}}$
$\newcommand{\UGent}{\affil{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}}$
$\newcommand{\IPARC}{\affil{Instituto de F\'{\i}sica de Part\'{\i}culas y del Cosmos IPARCOS, Facultad de Ciencias F\'{\i}sicas, Universidad Complutense de Madrid, E-28040, Spain}}$
$\newcommand{\STScI}{\affil{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\STScIESA}{\affiliation{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\McMaster}{\affil{Department of Physics and Astronomy, McMaster University, Hamilton, ON L8S 4M1, Canada}}$
$\newcommand{\INAF}{\affil{INAF -- Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50157, Firenze, Italy}}$
$\newcommand{\Sydney}{\affil{Sydney Institute for Astronomy, School of Physics A28, The University of Sydney, NSW 2006, Australia}}$
$\newcommand{\UA}{\affil{Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}}$
$\newcommand{\LERMA}{\affil{Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universit\'es, 75014 Paris}}$
$\newcommand{\SAIMSU}{\affil{Sternberg Astronomical Institute, Lomonosov Moscow State University, Universitetsky pr. 13, 119234 Moscow, Russia}}$
$\newcommand{\StockholmOKC}{\affil{The Oskar Klein Centre for Cosmoparticle Physics, Department of Physics, Stockholm University, AlbaNova, Stockholm, SE-106 91, Sweden}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\sfr}{M_{\odot} yr^{-1}}$
$\newcommand{\obstime}{112.6}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\Caltech$}{\affil{California Institute of Technology, 1200 E. California Blvd., MC 249-17, Pasadena, CA 91125, USA}}$
$\newcommand{$\Tamkang$}{\affil{Department of Physics, Tamkang University, No.151, Yingzhuan Rd., Tamsui Dist., New Taipei City 251301, Taiwan}}$
$\newcommand{\GEMINI}{\affil{Gemini Observatory/NSF’s NOIRLab, 950 N. Cherry Avenue, Tucson, AZ, 85719, USA$
$}}$
$\newcommand{\ASCL}{\affil{Astrophysics Source Code Librar$
$y, Michigan Technological University, 1400 Townsend Drive, Houghton, MI 49931}}$
$\newcommand{$\OSU$}{\affil{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}}$
$\newcommand{$\Alberta$}{\affil{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}}$
$\newcommand{$\ANU$}{\affil{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}}$
$\newcommand{$\IPARCOS$}{\affil{Instituto de F\'{\i}sica de Part\'{\i}culas y del Cosmos, Universidad Complutense de Madrid, E-28040 Madrid, Spain}}$
$\newcommand{$\IPAC$}{\affil{Caltech-IPAC, 1200 E. California Blvd. Pasadena, CA 91125, USA}}$
$\newcommand{$\Carnegie$}{\affil{Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}}$
$\newcommand{$\CCAPP$}{\affil{Center for Cosmology and AstroParticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}}$
$\newcommand{$\CfA$}{\affil{Center for Astrophysics \mid Harvard \& Smithsonian , 60 Garden Street, Cambridge, MA 02138, USA}}$
$\newcommand{$\CITEVA$}{\affil{Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}}$
$\newcommand{$\CNRS$}{\affil{CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}}$
$\newcommand{$\ESO$}{\affil{European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei M\"{u}nchen, Germany}}$
$\newcommand{$\Heidelberg$}{\affil{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}}$
$\newcommand{$\COOL$}{\affil{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}}$
$\newcommand{$\ICRAR$}{\affil{International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}}$
$\newcommand{$\IRAM$}{\affil{Institut de Radioastronomie Millimétrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d'Hères, France}}$
$\newcommand{$\IRAP$}{\affil{CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}}$
$\newcommand{$\UPS$}{\affil{Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}}$
$\newcommand{$\ITA$}{\affil{Universit\"{a}t Heidelberg, Zentrum f\"{u}r Astronomie, Institut f\"{u}r Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}}$
$\newcommand{$\IWR$}{\affil{Universit\"{a}t Heidelberg, Interdisziplin\"{a}res Zentrum f\"{u}r Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}}$
$\newcommand{$\JHU$}{\affil{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}}$
$\newcommand{$\Leiden$}{\affil{Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands}}$
$\newcommand{$\Maryland$}{\affil{Department of Astronomy, University of Maryland, College Park, MD 20742, USA}}$
$\newcommand{$\MPE$}{\affil{Max-Planck-Institut f\"{u}r extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}}$
$\newcommand{$\MPIA$}{\affil{Max-Planck-Institut f\"{u}r Astronomie, K\"{o}nigstuhl 17, D-69117, Heidelberg, Germany}}$
$\newcommand{$\Nagoya$}{\affil{Department of Physics, Nagoya University, Furo-cho, Chikusa-ku, Nagoya, Aichi 464-8602, Japan}}$
$\newcommand{$\NRAO$}{\affil{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}}$
$\newcommand{$\OAN$}{\affil{Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}}$
$\newcommand{$\ObsParis$}{\affil{Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France}}$
$\newcommand{$\Princeton$}{\affil{Department of Astrophysical Sciences, Princeton University, Princeton, NJ 08544 USA}}$
$\newcommand{$\UToledo$}{\affil{University of Toledo, 2801 W. Bancroft St., Mail Stop 111, Toledo, OH, 43606}}$
$\newcommand{$\Toulouse$}{\affil{Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}}$
$\newcommand{$\UBonn$}{\affil{Argelander-Institut f\"ur Astronomie, Universit\"at Bonn, Auf dem H\"ugel 71, 53121 Bonn, Germany}}$
$\newcommand{$\UChile$}{\affil{Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}}$
$\newcommand{$\UCM$}{\affil{Departamento de F\'{\i}sica de la Tierra y Astrof\'{\i}sica, Universidad Complutense de Madrid, E-28040 Madrid, Spain}}$
$\newcommand{$\UCSD$}{\affil{Center for Astrophysics and Space Sciences, Department of Physics,  University of California,\San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}}$
$\newcommand{$\ULyon$}{\affil{Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574,\F-69230 Saint-Genis-Laval, France}}$
$\newcommand{$\UMass$}{\affil{University of Massachusetts—Amherst, 710 N. Pleasant Street, Amherst, MA 01003, USA}}$
$\newcommand{$\UWyoming$}{\affil{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}}$
$\newcommand{\LAM}{\affil{$
$Aix Marseille Univ, CNRS, CNES, LAM (Laboratoire d’Astrophysique de Marseille),  F-13388 Marseille,$
$France}}$
$\newcommand{$\UHawaii$}{\affil{Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}}$
$\newcommand{$\UGent$}{\affil{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}}$
$\newcommand{$\IPARC$}{\affil{Instituto de F\'{\i}sica de Part\'{\i}culas y del Cosmos IPARCOS, Facultad de Ciencias F\'{\i}sicas, Universidad Complutense de Madrid, E-28040, Spain}}$
$\newcommand{$\STScI$}{\affil{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{$\STScI$ESA}{\affiliation{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{$\McMaster$}{\affil{Department of Physics and Astronomy, McMaster University, Hamilton, ON L8S 4M1, Canada}}$
$\newcommand{$\INAF$}{\affil{INAF -- Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50157, Firenze, Italy}}$
$\newcommand{$\Sydney$}{\affil{Sydney Institute for Astronomy, School of Physics A28, The University of Sydney, NSW 2006, Australia}}$
$\newcommand{$\UA$}{\affil{Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}}$
$\newcommand{$\LERMA$}{\affil{Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universit\'es, 75014 Paris}}$
$\newcommand{$\SAIMSU$}{\affil{Sternberg Astronomical Institute, Lomonosov Moscow State University, Universitetsky pr. 13, 119234 Moscow, Russia}}$
$\newcommand{$\StockholmOKC$}{\affil{The Oskar Klein Centre for Cosmoparticle Physics, Department of Physics, Stockholm University, AlbaNova, Stockholm, SE-106 91, Sweden}}$
$\newcommand{$\msun$}{M_{\odot}}$
$\newcommand{$\sfr$}{M_{\odot} yr^{-1}}$
$\newcommand{$\obstime$}{112.6}$</div>



<div id="title">

# S

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.02667-b31b1b.svg)](https://arxiv.org/abs/2212.02667)<mark>Appeared on: 2022-12-07</mark> - _Re-submitted after addressing minor comments from referee. To be published as part of PHANGS-JWST ApJL Focus Issue_

</div>
<div id="authors">

<mark><mark>Janice C. Lee</mark></mark>, et al. -- incl., <mark><mark>Eva Schinnerer</mark></mark>, <mark><mark>Frank Bigiel</mark></mark>, <mark><mark>Kiana F. Henny</mark></mark>, <mark><mark>Jonathan D. Henshaw</mark></mark>, <mark><mark>Annie Hughes</mark></mark>, <mark><mark>Kathryn Kreckel</mark></mark>, <mark><mark>Jing Li</mark></mark>, <mark><mark>Daizhong Liu</mark></mark>

</div>
<div id="abstract">

**Abstract:** The PHANGS collaboration has been building a reference dataset for the multi-scale, multi-phase study of star formation and the interstellar medium in nearby galaxies.  With the successful launch and commissioning of JWST, we can now obtain high-resolution infrared imaging to probe the youngest stellar populations and dust emission on the scales of star clusters and molecular clouds ($\sim$5-50 pc).  In Cycle 1, PHANGS is conducting an 8-band imaging survey from 2-21$\mu$m of 19 nearby spiral galaxies.CO(2--1) mapping, optical integral field spectroscopy, and UV-optical imaging for all 19 galaxies have been obtained through large programs with ALMA, VLT/MUSE, and Hubble. PHANGS-JWST enables a full inventory of star formation, accurate measurement of the mass and age of star clusters, identification of the youngest embedded stellar populations, and characterization of the physical state of small dust grains.  When combined with Hubble catalogs of$\sim$10,000 star clusters, MUSE spectroscopic mapping of$\sim$20,000 HII regions, and$\sim$12,000 ALMA-identified molecular clouds, it becomes possible to measure the timescales and efficiencies of the earliest phases of star formation and feedback, build an empirical model of the dependence of small dust grain properties on local ISM conditions, and test our understanding of how dust-reprocessed starlight traces star formation activity, all across a diversity of galactic environments.  Here we describe the PHANGS-JWST Treasury survey, present the remarkable imaging obtained in the first few months of science operations, and provide context for the initial results presented in the first series of PHANGS-JWST publications.

</div>

<div id="div_fig1">

<img src="tmp_2212.02667/./Figures/sed_model_filter.png" alt="Fig1" width="100%"/>

**Figure 1. -** \textbf{top}: Model SEDs of dust-enshrouded young star clusters with filter coverage of the PHANGS-HST and JWST Treasury programs. Models are of a solar metallicity $10^4$ M$_\odot$ young star cluster at $d=10$ Mpc with varying ages and levels of extinction generated with CIGALE \citep[][]{boquien19,turner21}. Colored bands show the 8 JWST NIRCAM and MIRI filters, spanning from 2$-21\mu$m, selected to probe stellar photospheric emission, PAH features, and the dust continuum. The 5 UV-visible filters used by PHANGS-HST \citep{phangs-hst} appear in grey. \textbf{bottom}: Normalized throughput for each of the eight filters used in the PHANGS-JWST survey.  For comparison, the four Spitzer IRAC filters and MIPS 24 $\micron$ filter are also shown.  A model dust SED is overplotted to illustrated the features probed by each filter.  The three PAH features targeted by our survey are marked. (*fig:filters*)

</div>
<div id="div_fig2">

<img src="tmp_2212.02667/./Figures/judy_n628.png" alt="Fig7.1" width="50%"/><img src="tmp_2212.02667/./Figures/judy_ic5332.jpg" alt="Fig7.2" width="50%"/>

**Figure 7. -** Color composite images produced by J. Schmidt from MIRI data of NGC 628 (top) and IC 5332 (bottom). Red: MIRI F2100W; orange: MIRI F1130W; cyan: MIRI F770W and extra overall brightness in grayscale: MIRI F1000W.  The images were used as the models for the first two ESA/JWST Pictures of the Month from the JWST Cycle 1 General Observers Program as featured at \url{https://esawebb.org/images/potm2208a/} and \url{https://esawebb.org/images/comparisons/potm2209a/}. (*fig:judy1*)

</div>
<div id="div_fig3">

<img src="tmp_2212.02667/./Figures/gri_ha_v4.png" alt="Fig2" width="100%"/>

**Figure 2. -** Composite $gri$+H$\alpha$+CO images for the 19 galaxies in the PHANGS-JWST Treasury program. The $gri$+H$\alpha$ images are constructed from VLT MUSE full-field spectral mapping \citep{phangs-muse}, with H$\alpha$ line emission in red, and combined with ALMA CO(2--1) \citep{phangs-alma} maps, with CO(2--1) flux in blue. JWST NIRCam and MIRI footprints have been defined to overlap existing MUSE, ALMA, and HST data. Footprints shown with dashed lines represent observations that have not been executed at time of writing (October 2022), and may rotate slightly due to the range of allowed orient angles specified for the target. The JWST sample spans a factor of $\sim50$ in stellar mass and SFR, and a factor of $\sim100$ in CO surface density (Table \ref{tab:galaxysample2}). It includes galaxies with prominent dust lanes, a full range of stellar bar, bulge, spiral arm morphologies, as well as nuclear starbursts and AGN activity. (*fig:footprints*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

127  publications in the last 7 days.
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/html/2211.16996.md
	 _build/html/2211.16510.md
	 _build/html/2211.16123.md
	 _build/html/2211.15816.md
	 _build/html/2211.15698.md
	 _build/html/2211.15576.md
	 _build/html/2211.15434.md
	 _build/html/2211.15288.md
	 _build/html/2211.15224.md
	 _build/html/2211.14543.md
	 _build/html/2211.14330.md
	 _build/html/2211.14322.md
	 _build/html/2211.14132.md
	 _build/html/2211.13811.md
	 _build/html/2211.13426.md
	 _build/html/2211.13318.md
	 _build/html/2211.13146.md
	 _build/html/2211.12613.md
	 _build/html/2211.11767.md
	 _build/html/2211.09840.md
	 _build/html/2211.09217.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers